In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout

In [ ]:
model_name = "rnnv6fun"

In [ ]:
# Generowanie przykładowych danych
# Wygenerowanie wartości X
X = np.arange(0, 151).reshape(-1, 1)
# Wygenerowanie wartości y
# y = X
# y = X + np.random.randn(len(X),1)
y = np.sin(X/40*np.pi) + np.cos((X - 10)/20*np.pi)
# Przeskalowanie wartości y do zakresu od 0 do 1
min_y = np.min(y)
max_y = np.max(y)
y = (y - min_y) / (max_y - min_y)
plt.plot(y, color='black', label='Data', marker='.')
plt.savefig(f'data_{model_name}.png')
plt.show()

In [ ]:
# Funkcja do przygotowania sekwencji danych
def create_sequences(y_data, seq_length):
    X_sequences = []
    y_labels = []
    for i in range(len(y_data) - seq_length):
        X_sequences.append(y_data[i:i+seq_length])
        y_labels.append(y_data[i+seq_length])
    return np.array(X_sequences), np.array(y_labels)

In [ ]:
# Długość sekwencji
seq_length = 10

In [ ]:
# Tworzenie sekwencji
X_seq, y_seq = create_sequences(y, seq_length)
print(f'Kształt X_seq: {X_seq.shape}, Kształt y_seq: {y_seq.shape}')

In [ ]:
# Podział danych na zbiór treningowy i testowy
split_index = round(0.8 * len(X_seq))
X_train, X_test = X_seq[:split_index], X_seq[split_index:]
y_train, y_test = y_seq[:split_index], y_seq[split_index:]
print(f'Kształt X_train: {X_train.shape}, Kształt y_train: {y_train.shape}')
print(f'Kształt X_test: {X_test.shape}, Kształt y_test: {y_test.shape}')

In [ ]:
# Definicja modelu
Dropout_rate=0
model = Sequential()
model.add(SimpleRNN(32, input_shape=(None, 1), activation='tanh',return_sequences = False))  # Warstwa SimpleRNN z 10 neuronami
model.add(Dense(1))  # Warstwa wyjściowa z jednym neuronem

In [ ]:
# Kompilacja modelu
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Trenowanie modelu
batch_size = round(len(X_train)/5)
print('batch_size =',batch_size)
history = model.fit(X_train, y_train, epochs=200, batch_size=batch_size, verbose=1)

In [ ]:
# Plotting Loss vs Epochs
plt.figure(figsize =(10,7))
plt.plot(history.history["loss"])
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.title("Simple RNN model, Loss vs Epoch")
plt.ylim(0, 0.1)  # Ustawienie zakresu osi y od 0 do 0.1
mid_x = len(history.history["loss"]) / 2  # Środek osi x (liczba epok podzielona przez 2)
mid_y = 0.05  # Środek osi y (0.05 to połowa zakresu od 0 do 0.1)
plt.text(mid_x, mid_y, f' Sequence length = {seq_length} \n Batch size={batch_size} \n Dropout rate={Dropout_rate} \n Input data - function \n Final MSE = {round(history.history["loss"][-1],5)}', fontsize=16, ha='left')# Dodanie tekstu na środku obrazka
plt.savefig(f'history_{model_name}.png')
plt.show()

In [ ]:
# Przewidywania na danych treningowych
predictions_train = model.predict(X_train)
# wykres
plt.plot(predictions_train.flatten(), color='red', label='Predykcje')
plt.plot(y_train.flatten(), color='blue', label='Dane treningowe',alpha=0.5)
plt.legend()
plt.savefig(f'pred_train_{model_name}.png')
plt.show()

In [ ]:
# Przewidywania na danych testowch
predictions_test = model.predict(X_test)
# wykres
plt.plot(predictions_test.flatten(), color='red', label='Predykcje',alpha=1)
plt.plot(y_test.flatten(), color='black', label='Dane testowe',alpha=0.5)
plt.legend()
plt.savefig(f'pred_test_{model_name}.png')
plt.show()

In [ ]:
# export modelu RNN
model.save(f"keras_{model_name}.keras")

In [ ]:
# Wizualizacja modelu
from keras.utils import plot_model
plot_model(model, to_file=f'architecture_{model_name}.png', show_shapes=True, show_layer_names=True)